In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)
import platform

Versão do TensorFlow:  2.16.1


In [17]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [18]:
os = platform.system()
path = './angle_datas.pkl'
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model = "./models/angle/model_saved.h5"
save_model_keras = "./models/angle/model_saved.keras"

if os == 'Darwin':  # 'Darwin' é o nome do kernel do macOS
    path = '../../datas/angle_datas.pkl'
    save_model = "../../models/angle/model_saved.h5"
    save_model_keras = "../../models/angle/model_saved.keras"

In [19]:


# Carregar os dados do arquivo .pkl
with open(path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [20]:
%load_ext tensorboard
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(3, activation='softmax')  #  
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.array(X_train), np.array(y_train), epochs=1000, batch_size=10000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/1000


/opt/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step - accuracy: 0.5068 - loss: 12.4512 - val_accuracy: 0.4848 - val_loss: 6.7225
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5057 - loss: 6.4787 - val_accuracy: 0.4784 - val_loss: 0.6908
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5030 - loss: 0.6597 - val_accuracy: 0.9805 - val_loss: 0.0515
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9881 - loss: 0.0409 - val_accuracy: 0.9805 - val_loss: 0.0404
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9881 - loss: 0.0291 - val_accuracy: 0.9805 - val_loss: 0.0368
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9881 - loss: 0.0252 - val_accuracy: 0.9805 - val_loss: 0.0349
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9881 - loss: 0.0233 - val_accuracy: 0.9805 - val_loss: 0.0339
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9881 - loss: 0.0222 - val_accuracy: 0.9805 - val_loss: 0.

In [21]:
# Avaliar o modelo
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Model Accuracy: {accuracy * 100:.2f}%')

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 1.0098e-04
Model Accuracy: 100.00%


In [26]:
# Ações
# 0 Nothing
# 1 Girar para a Esquerda
# 2 Girar para a Direita

# GO TO RIGHT
angle = 180
 
class_names = ['Nothing', 'RoteteLeft', 'RotateRight', ]
predictions = model.predict(np.array([ angle]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)
 



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1-Must be GO TO RIGHT: RoteteLeft - with a 57.61 percent confidence.


In [23]:
%tensorboard

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [24]:
model.save(save_model)
model.save(filepath=save_model_keras)

### From here to you only MACOS to test 

In [25]:

# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('../../models/angle/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, '../../models/angle/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model '../../models/angle/saved_model/' ../../models/tf/angle/web_model

INFO:tensorflow:Assets written to: ../../models/angle/saved_model/assets


INFO:tensorflow:Assets written to: ../../models/angle/saved_model/assets
